<a href="https://colab.research.google.com/github/ethvedbitdesjan/SummerResearch/blob/main/ViTMAELastLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install datasets transformers
!pip install wandb -qU
!pip install weightwatcher
!pip install accelerate -U

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: vedaantj. Use `wandb login --relogin` to force relogin


True

In [3]:
%env WANDB_PROJECT=Funny_Not_Funny

env: WANDB_PROJECT=Funny_Not_Funny


In [4]:
wandb.init(
    # set the wandb project where this run will be logged
    project='Funny_Not_Funny',
    name='ViTMAE_Large_Last_Layer'
)

In [5]:
from google.colab import drive
drive.mount('content/')

Drive already mounted at content/; to attempt to forcibly remount, call drive.mount("content/", force_remount=True).


In [6]:
%cd content/MyDrive/SummerResearch

/content/content/MyDrive/SummerResearch


In [7]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="Dataset/")

Resolving data files:   0%|          | 0/1615 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/539 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/539 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
# Critical imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import random
import cv2
import copy
import torch
import torchvision
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import weightwatcher as ww

In [9]:
from transformers import AutoImageProcessor
model_name_or_path = 'facebook/vit-mae-large'
image_processor  = AutoImageProcessor.from_pretrained(model_name_or_path)
image_processor

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [10]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = image_processor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    return inputs

prepared_ds = dataset.with_transform(transform)

In [11]:
import numpy as np
from datasets import load_metric

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

<ipython-input-11-223a1e4a98a9>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [12]:
from transformers import AutoImageProcessor, ViTForImageClassification

labels = prepared_ds['train'].features['label'].names
model = ViTForImageClassification.from_pretrained(model_name_or_path, num_labels=len(labels), id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}, ignore_mismatched_sizes=True)

NUM_CLASSES = 2
BATCH_SIZE = 8
device = 'cuda' if torch.cuda.is_available() else 'cpu'

You are using a model of type vit_mae to instantiate a model of type vit. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at facebook/vit-mae-large were not used when initializing ViTForImageClassification: ['decoder.decoder_layers.1.output.dense.weight', 'decoder.decoder_layers.0.attention.attention.value.weight', 'decoder.decoder_layers.0.layernorm_after.weight', 'decoder.decoder_layers.2.attention.attention.key.weight', 'decoder.decoder_layers.5.output.dense.bias', 'decoder.decoder_layers.2.attention.attention.query.weight', 'decoder.decoder_layers.4.layernorm_before.weight', 'decoder.decoder_layers.6.layernorm_before.bias', 'decoder.decoder_layers.3.attention.attention.query.bias', 'decoder.decoder_layers.6.attention.output.dense.bias', 'decoder.decoder_layers.4.attention.attention.value.bias', 'decoder.decoder_layers.2.attention.output.dense.bias', 'decoder.decoder_layers.2.layernorm_before.weight', 'decoder.decoder

In [23]:
for param in model.vit.parameters():
  param.requires_grad = False
for param in model.parameters():
  print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [26]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vitmae-large-last-layer",
  per_device_train_batch_size=BATCH_SIZE,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=10,
  learning_rate=1e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='wandb',
  load_best_model_at_end=True,
  optim='adamw_torch',
  weight_decay=0.001
)

In [27]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=image_processor,
)

In [28]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Step,Training Loss,Validation Loss,Accuracy
200,0.646900,0.684167,0.541744
400,0.674300,0.672486,0.549165
600,0.621500,0.665723,0.564007
800,0.643900,0.664487,0.562152


***** train metrics *****
  epoch                    =          4.0
  total_flos               = 1648082213GF
  train_loss               =         0.66
  train_runtime            =   0:09:20.35
  train_samples_per_second =       11.528
  train_steps_per_second   =        1.442


In [29]:
watcher = ww.WeightWatcher(model=model)
details = watcher.analyze()
summary = watcher.get_summary(details)
summary

{'log_norm': 3.549,
 'alpha': 4.580152887150201,
 'alpha_weighted': 7.685950343877592,
 'log_alpha_norm': 8.284536189659663,
 'log_spectral_norm': 1.735,
 'stable_rank': 77.6}

In [30]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.5622
  eval_loss               =     0.6645
  eval_runtime            = 0:00:34.94
  eval_samples_per_second =     15.425
  eval_steps_per_second   =      1.946


In [31]:
metrics = trainer.predict(prepared_ds['test'])
trainer.log_metrics("eval", metrics.metrics)
trainer.save_metrics("eval", metrics.metrics)
wandb.finish()

***** eval metrics *****
  test_accuracy           =     0.5918
  test_loss               =     0.6528
  test_runtime            = 0:00:32.11
  test_samples_per_second =     16.783
  test_steps_per_second   =      2.117


eval/accuracy,▁▃█▇▇
eval/loss,█▄▁▁▁
eval/runtime,▄█▂▁▆
eval/samples_per_second,▄▁▇█▃
eval/steps_per_second,▄▁▇█▃
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▇▆▇█▅▄▅▄▄▄▆▄▆▅▂▂▅▅▄▁▄▄▄▅▁▂▅▄▃▅▃▃▂▃▅▁▃▆▄
train/total_flos,▁
train/train_loss,▁


In [32]:
predictions_tensor = torch.tensor(metrics.predictions)
preds = torch.sigmoid(predictions_tensor)
print(preds.shape)
preds = torch.argmax(preds, dim=1)
preds, preds.shape

torch.Size([539, 2])


(tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
         0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1

In [33]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [34]:
print(accuracy_score(preds, metrics.label_ids), f1_score(preds, metrics.label_ids))
pd.DataFrame(confusion_matrix(preds, metrics.label_ids), columns=['Test Funny', 'Test Not Funny'], index=['Pred Funny', 'Pred Not Funny'])

0.5918367346938775 0.7349397590361446


,Test Funny,Test Not Funny
Pred Funny,14,9
Pred Not Funny,211,305
